# Experiment 05-09

In [1]:
import sys
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information
import logging
from pqdm.threads import pqdm
from tqdm.notebook import tqdm

## Configure Logging

In [2]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("ex05_09.log"),
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2022-07-15 13:01:52 INFO     Started


## Show the available Platforms

In [3]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: Apple
Platform 0 - Vendor: Apple
Platform 0 - Version: OpenCL 1.2 (Apr 19 2022 18:44:44)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Apple M1
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1000 Mhz
 Device - Compute Units: 8
 Device - Local Memory: 32 KB
 Device - Constant Memory: 1048576 KB
 Device - Global Memory: 5 GB
 Device - Max Buffer/Image Size: 1024 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [4]:
platform_number = 0
device_number = 0

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

In [5]:
cl_device = cl_devices[device_number]

cl_threads = cl_device.max_compute_units * cl_device.max_work_group_size
if cl_device.type & 4 == 0:
    cl_threads = cl_devices.max_work_group_size

print('Max Compute Units:', cl_device.max_compute_units)
print('Max Compute Units:', cl_device.max_work_group_size)
print('Threads:', cl_threads)

Max Compute Units: 8
Max Compute Units: 256
Threads: 2048


## Compile the Program

In [6]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [7]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: hash_main;get_single_hash;get_random_numbers;get_random_string;get_single_hash_nonce;mine_eight;mine_nonce;mine_eight_sequential;mine_nonce_sequential


## Mine Nonce

In [8]:
max_nonce = 20
max_output_size = 64

In [9]:
def display_output(zeros_found : dict, nonce : np.ndarray,
                   nonce_len : np.ndarray) -> None:
    """
    Display the output generated by the OpenCL Kernel.
    """
    for i in range(0, 64):
        if nonce_len[i] > 0:
            if (i not in zeros_found):
                nonce_str = nonce[i * max_nonce:i * max_nonce + nonce_len[i]].tobytes().decode('UTF-8')
                zeros_found[i] = nonce_str
                hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))

                logging.info("%4d: [%2d] %16s %64s" % (i, nonce_len[i], nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))

In [10]:
print(np.iinfo(np.unsignedinteger).max)
print(np.iinfo(np.ulonglong).max)
print(len(str(np.iinfo(np.ulonglong).max)))

18446744073709551615
18446744073709551615
20


In [11]:
#cl_threads = 1

In [12]:
postfix = ""
base_text = 'this is a description of the latest block'
plaintext = base_text + postfix
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)

# set up the variables to generate the random numbers
seed = np.ulonglong(0)
window_size = np.uint32(10000)
nonce = np.zeros(shape=max_nonce * max_output_size, dtype=np.uint8)
nonce_len = np.zeros(shape=max_output_size, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)

zeros_found = {}

keep_running = True
while (keep_running):
    cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)

    # execute the program
    cl_program.mine_nonce_sequential(
        cl_queue, (cl_threads,), None,
        cl_seed,
        cl_window_size,
        cl_plaintext_bytes,
        cl_plaintext_length,
        cl_nonce,
        cl_nonce_len
    )

    # get the results
    cl.enqueue_copy(cl_queue, nonce, cl_nonce)
    cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)

    # display the last nonce tried for each thread
    # for i in range(0, cl_threads):
    #     logging.info("Thread %d: %d" % (i, last_nonce[i]))

    # for i in range(0, nonce.size):
    #     print(nonce[i], end=' ')

    # display the results
    display_output(zeros_found, nonce, nonce_len)
    #logging.info('Iteration Complete.')

    seed = np.ulonglong(seed + cl_threads * window_size)
    keep_running = True

2022-07-15 13:01:56 INFO        1: [ 8]         19500000 e3a63c89b4580d11310bf9b4291fc2a6069d535b247805bfd499abc5bd94886c
2022-07-15 13:01:56 INFO        2: [ 7]          1568000 5c689ad91aaa07bff18c99f90df27e31ec6982fe68487abf5e3149b434d03936
2022-07-15 13:01:56 INFO        3: [ 7]          9370001 000c8b299811fb42a75c9593bf7daa6a71f672c38353d0621b95a9b96d436561
2022-07-15 13:01:56 INFO        4: [ 8]         19110027 00005aeab97ce13b54bbb762b8654099a496cc98877eefff10760576a45fbb98
2022-07-15 13:01:56 INFO        5: [ 7]          1140060 000004fcb1a7306741474bc477709f877ff9d014d10be9b40c0ede692fc8cacd
2022-07-15 13:01:56 INFO        6: [ 8]         11852736 00000012658436c4c2e6be5ec0960b6d9170ed284a46aa7f5a48ddd4aa1a5bc6


In [ ]:
input_buffer = {}
nonce = {}
nonce_len = 0
seed = 123456

while (seed > 0):
    nonce_len += 1
    input_buffer[nonce_len - 1] = (seed % 10)
    seed = seed // 10

print(input_buffer)

for i in range(0, nonce_len):
    nonce[nonce_len-1 - i] = input_buffer[i]

for i in range(0, nonce_len):
    print(nonce[i])
    